In [106]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model, load_model, model_from_json
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np
import sys
import h5py
import os
import tensorflow as tf
from sklearn.metrics import accuracy_score, log_loss, f1_score
# tf.enable_eager_execution()

from tensorflow.nn.rnn_cell import LSTMStateTuple
from tensorflow.keras.layers import TimeDistributed
from tensorflow.contrib.rnn import OutputProjectionWrapper

import torch
from torch.autograd import Variable
from torch import optim,nn
from torch.utils.data import TensorDataset, DataLoader
from ignite.metrics import Accuracy, Loss
import pickle
from scipy.stats import pearsonr

In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


In [4]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [5]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [6]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [7]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [8]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'h', 'a', 'i', 'č', 'ů', 'ó', '5', 'ž', 'ú', 'á', '7', 'š', 'y', 'ť', 'ě', 'ď', '0', 'u', '6', 'v', 'g', '.', 'END', 'ř', 't', 'z', '1', 'x', '3', '9', 'o', 'b', 'd', 'l', 'ý', 'm', 'f', 'n', 'e', 'p', '8', '4', 'q', '2', ' ', 'c', 'ň', 's', 'j', 'é', 'w', 'k', 'r', 'í'}
vocab length is  54
length of data_set is  15226


In [9]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [10]:
# with open('char_index.txt', 'wb') as handle:
#     pickle.dump(char_index, handle)

with open('char_index.txt', 'rb') as handle:
    char_index = pickle.loads(handle.read())


In [11]:
print(char_index)

{'h': 14, 'ň': 47, 'a': 45, 'i': 32, 'ů': 23, 'ó': 52, 'b': 25, '5': 40, 'ú': 26, 'á': 17, '7': 2, 'š': 36, 'y': 50, 'ť': 46, 'č': 38, 'ě': 24, 'ď': 13, '3': 49, '0': 44, 'u': 30, 'v': 19, '.': 34, '6': 12, 'END': 43, 'ř': 31, 't': 48, 'z': 3, '1': 41, 'x': 15, 'é': 20, '9': 7, 'o': 37, 'ý': 11, 'g': 9, 'l': 0, 'f': 27, 'n': 51, 'e': 1, 'p': 5, '8': 53, '4': 29, 'd': 16, 'q': 10, '2': 18, ' ': 22, 'c': 42, 'ž': 39, 's': 8, 'j': 21, 'w': 33, 'm': 28, 'k': 35, 'r': 6, 'í': 4}


In [12]:
#train test split
np.random.seed(0)
msk = np.random.rand(len(data_set1)) < 0.9
train = data_set1[msk]
test = data_set1[~msk]     


In [13]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [14]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [15]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_x = []
train_y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_x.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_y.append([1, 0])
    else:
        train_y.append([0, 1])
    
train_x=np.asarray(train_x)
train_y=np.asarray(train_y)

In [16]:
test_x = []
test_y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_x.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_y.append([1, 0])
    else:
        test_y.append([0, 1])
    
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [17]:
print(np.asarray(train_x).shape)
print(np.asarray(test_y).shape)

(13713, 30, 54)
(1513, 2)


In [18]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in czech.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in czech.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

### Making BatchLoader

In [19]:
#Creating Tensor Datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#Now creating Data Loaders
batch = 600

train_loader = DataLoader(train_data, shuffle = True, batch_size = batch)
test_loader = DataLoader(test_data, shuffle = True, batch_size = batch)

dataiter = iter(train_loader)

sample_x, sample_y = dataiter.next()

print("Sample input size of one instance from iter is: ", sample_x.size()) #Batch size, Sequence Length, Feature Dimension
print("Sample target size of that same one instance from iter is", sample_y.size())

Sample input size of one instance from iter is:  torch.Size([600, 30, 54])
Sample target size of that same one instance from iter is torch.Size([600, 2])


#### build model in Tensorflow ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [20]:
''' Parameters '''
tf.reset_default_graph()
no_units=524 

features=train_x.shape[-1] #30
# new_phoneme=new_phoneme.reshape((-1,1,features))

### Making Placeholders

In [21]:
train = tf.placeholder(dtype = tf.float32, shape = [None, None , features], name = 'input')
target = tf.placeholder(dtype = tf.int32, shape = [None, 2], name = 'label')

In [22]:
encoder_fw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]
encoder_bw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]

for i in range(1,3):

    encoder_fw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))
    encoder_bw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))


In [23]:


encoder_outputs, encoder_fw_state, encoder_bw_state = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                                           cells_fw = encoder_fw,
                                                                           cells_bw = encoder_bw,
                                                                           inputs = train,
                                                                           dtype = tf.float32
                                                                          )

In [24]:
# encoder_outputs = tf.concat((encoder_fw_output, encoder_bw_output), 2)

encoder_state_c = tf.concat((encoder_fw_state[-1][0].c, encoder_bw_state[-1][0].c), 1)

encoder_state_h = tf.concat((encoder_fw_state[-1][0].h, encoder_bw_state[-1][0].h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_state_c,
    h=encoder_state_h
)

In [25]:
output=tf.layers.Dense(2)(encoder_state_h)

In [26]:
losss= tf.losses.softmax_cross_entropy(target, output)
trainop =  tf.train.AdamOptimizer(learning_rate=0.001).minimize(losss)



In [36]:
losx=[]
saver = tf.train.Saver()
# with tf.device('/gp):
# with tf.device('/gpu:0'):
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    if(tf.train.checkpoint_exists("tmp/model.ckpt")):
        saver.restore(sess, "tmp/model.ckpt")
        print("Model Restored")

    for i in range(15):
        for batch_j, target_j in train_loader:

            loss,_=sess.run([losss,trainop],feed_dict={train: batch_j,
                                                       target : target_j})
            losx.append([loss])
        print("Current epoch going is...", i," and current loss is...",loss)
    save_path = saver.save(sess, "tmp/model_tensorflow.ckpt")
    print("Model saved in path: %s" % save_path)

    



Current epoch going is... 0  and current loss is... 0.6181861
Current epoch going is... 1  and current loss is... 0.5131525
Current epoch going is... 2  and current loss is... 0.45227897
Current epoch going is... 3  and current loss is... 0.4561371
Current epoch going is... 4  and current loss is... 0.4264607
Current epoch going is... 5  and current loss is... 0.3748216
Current epoch going is... 6  and current loss is... 0.38874
Current epoch going is... 7  and current loss is... 0.36775187
Current epoch going is... 8  and current loss is... 0.2857654
Current epoch going is... 9  and current loss is... 0.2826659
Current epoch going is... 10  and current loss is... 0.3081251
Current epoch going is... 11  and current loss is... 0.30422148
Current epoch going is... 12  and current loss is... 0.27230778
Current epoch going is... 13  and current loss is... 0.27890605
Current epoch going is... 14  and current loss is... 0.27448007
Model saved in path: tmp/model_tensorflow.ckpt


In [50]:
new_ta = tf.placeholder(dtype= tf.float32, shape = [None, None])

In [109]:
accc = []
# with tf.device('/gpu:0'):
with tf.Session() as sess:

#         sess.run(tf.global_variables_initializer())
    if(tf.train.checkpoint_exists("tmp/model_tensorflow.ckpt")):
        saver.restore(sess, "tmp/model_tensorflow.ckpt")
        print("Model Restored")
    for batch_j, target_j in test_loader:
        out = sess.run([output], feed_dict={train: batch_j})
        out = np.squeeze(out)
#             out = np.round(out)
#         print("This is",sum([np.argmax(out[i])==np.argmax(target_j[i].numpy()) for i in range(target_j.shape[0])])/1000
        metric = accuracy_score(np.argmax(target_j.numpy(), axis= 1),  np.argmax(np.squeeze(out), axis = 1))
        print("real score",metric )

#         acc = tf.keras.backend.categorical_crossentropy(new_ta, out)
        something = metric
        something = np.squeeze(something)
        accc.append(np.mean(something))
    print("Final mean is: ",np.mean(accc))

INFO:tensorflow:Restoring parameters from tmp/model_tensorflow.ckpt
Model Restored
real score 0.8766666666666667
real score 0.8733333333333333
real score 0.865814696485623
Final mean is:  0.8719382321618744


In [110]:
name=["Martin", "Martini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j.lower()]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
with tf.Session() as sess:

#         sess.run(tf.global_variables_initializer())
    if(tf.train.checkpoint_exists("tmp/model_tensorflow.ckpt")):
        saver.restore(sess, "tmp/model_tensorflow.ckpt")
    out = sess.run([output], feed_dict={train: np.asarray(X)})
    out = np.squeeze(out)
        
out = np.argmax(out, axis=1)
for i in range(out.shape[0]):
    if out[i] == 0:
        print(name[i], "....it's male")
    else:
        print(name[i],"....It's female")
# pred=model.predict(np.asarray(X))
# print(np.round(out[:,-1,:]))

INFO:tensorflow:Restoring parameters from tmp/model_tensorflow.ckpt
Martin ....it's male
Martini ....It's female
